# scrapy 汽车之家scrapy工作原理

搜索汽车之家->报价->宝马

https://car.autohome.com.cn/price/brand-15.html

获取页面中汽车名字

## 3. scrapy 架构组成

```plaintext
(1) 引擎                -> 自动运行，无需关注，会自动组织所有的请求对象，分发给下载器

(2) 下载器              -> 从引擎处获取到请求对象后，请求数据

(3) spiders             -> Spider类定义了如何爬取某个（或某些）网站。包括了爬取的动作（例如：是否跟进链接）以及如何从网页的内容中提取结构化数据（爬取item）。换句话说，Spider就是您定义爬取的动作及分析某个网页的地方。

(4) 调度器              -> 有自己的调度规则，无需关注

(5) 管道（Item pipeline）-> 最终处理数据的管道，会预留接口供我们处理数据

当Item在Spider中被收集之后，它将会被传递到Item Pipeline，一些组件会按照一定的顺序执行对Item的处理。
每个Item pipeline组件（有时称之为“Item Pipeline”）是实现了简单方法的Python类。他们接收到Item并通过它执行一些行为，同时也决定此Item是否继续通过pipeline，或是被丢弃而不再进行处理。

以下是item pipeline的一些典型应用：
        1. 清理HTML数据
        2. 验证爬取的数据（检查item包含某些字段）
        3. 查重并丢弃重复内容
        4. 将爬取结果保存到数据库中

```

## 4. Scrapy工作原理

1. 引擎向 Spiders 要url
2. 引擎将要爬取的url交给调度器
3. 调度器会将url生成请求对象放入到指定的队列中
4. 从队列中取出一个请求
5. 引擎将请求交给下载器
6. 下载器发送请求获取互联网数据
7. 下载器将数据返回给引擎
8. 引擎将数据交给spiders
9. spiders通过xpath解析数据，得到数据或url
10. spiders将数据或者url交给引擎
11. 引擎判断该数据还是url，是数据，交给管道（item pipeline）处理，是url，交给调度器处理

![Scrapy工作原理](043_scrapy_scrapy工作原理.png "scrapy工作原理")

In [ ]:
# 创建项目
scrapy startproject scrapy_carhome_043

cd scrapy_carhome_043/scrapy_carhome_043/spiders

# 创建爬虫文件
scrapy genspider car https://car.autohome.com.cn/price/brand-15.html

In [ ]:
# scrapy_carhome_043/scrapy_carhome_043/spiders/car.py
# 运行 scrapy crawl car

import scrapy


class CarSpider(scrapy.Spider):
    name = "car"
    allowed_domains = ["car.autohome.com.cn"]
    # 注意如果你的请求的接口是html结尾的，那么是不需要加/的
    start_urls = ["https://car.autohome.com.cn/price/brand-15.html"]

    def parse(self, response):
        print("================")


In [ ]:
import scrapy


class CarSpider(scrapy.Spider):
    name = "car"
    allowed_domains = ["car.autohome.com.cn"]
    # 注意如果你的请求的接口是html结尾的，那么是不需要加/的
    start_urls = ["https://car.autohome.com.cn/price/brand-15.html"]

    def parse(self, response):
        # selector 列表
        name_list = response.xpath('//div[@class="main-title"]/a/text()')

        for name in name_list:
            print(name) # selector 对象
            print(name.extract()) # selector 对象 data的数据

In [ ]:
import scrapy


class CarSpider(scrapy.Spider):
    name = "car"
    allowed_domains = ["car.autohome.com.cn"]
    # 注意如果你的请求的接口是html结尾的，那么是不需要加/的
    start_urls = ["https://car.autohome.com.cn/price/brand-15.html"]

    def parse(self, response):
        # selector 列表
        name_list = response.xpath('//div[@class="main-title"]/a/text()')
        price_list = response.xpath('//div[@class="main-lever"]//span/span/text()')

        for i in range(len(name_list)):
            name = name_list[i].extract()
            price = price_list[i].extract()
            print(name, price)